<a href="https://colab.research.google.com/github/LiborioCastaneda99/Actividad-de-la-unidad-3-TOPICOS/blob/main/Actividad_de_la_unidad_3_Liborio_Casta%C3%B1eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Título:**
Aplicación de técnicas de ciencia de datos en la predicción de precios de viviendas

**Objetivo:**
El objetivo de esta actividad es que los estudiantes apliquen técnicas de ciencia de datos para predecir los precios de viviendas basándose en un conjunto de datos de entrenamiento.

**Solución:**
1. Link del conjunto de datos: /content/drive/MyDrive/Kaggle/housing.csv

2. Importa las bibliotecas necesarias al cuaderno de Colab:

In [ ]:
#importamos las librerias principales
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


2. Carga el conjunto de datos y realiza una exploración inicial:

In [ ]:
# Cargar el conjunto de datos
url = '/content/drive/MyDrive/Kaggle/housing.csv'
data = pd.read_csv(url)

#muestro los primeros 15 registros
print(data.head(15))

#muestro los ultimos 15 registros
print(data.tail(15))

#muestro información general de los datos
#principales estadíasticas
print(data.describe())

#Antes de analizar la correlación de las variables.
#Creo una nueva variable que contenga la relación de dormitorios por espacios de la casa
data['bed_rooms_per_room'] = data['total_bedrooms']/data['total_rooms']
atributos = ['median_house_value','median_income','total_rooms','housing_median_age','total_bedrooms','bed_rooms_per_room']


    longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0     -122.23     37.88                41.0        880.0           129.0   
1     -122.22     37.86                21.0       7099.0          1106.0   
2     -122.24     37.85                52.0       1467.0           190.0   
3     -122.25     37.85                52.0       1274.0           235.0   
4     -122.25     37.85                52.0       1627.0           280.0   
5     -122.25     37.85                52.0        919.0           213.0   
6     -122.25     37.84                52.0       2535.0           489.0   
7     -122.25     37.84                52.0       3104.0           687.0   
8     -122.26     37.84                42.0       2555.0           665.0   
9     -122.25     37.84                52.0       3549.0           707.0   
10    -122.26     37.85                52.0       2202.0           434.0   
11    -122.26     37.85                52.0       3503.0           752.0   
12    -122.2

3. Preparación y transformación de los datos.

In [ ]:
#reviso datos faltantes
data.isnull().sum()

print("Registros totales" , len(data))

data = data.dropna()
print("Registros luego de eliminar nulos" , len(data))


Registros totales 20640
Registros luego de eliminar nulos 20433


4. Utilizaremos un Pipeline de Skalearn para realizar la transformaciones secuenciales en los datos.

In [ ]:
#se procede a realizar un Encoding de la varible 'ocean_proximity'
#para eso utilizamos OneHotEncoder de la librería Sklearn
!pip install -U scikit-learn

from sklearn.preprocessing import OneHotEncoder, StandardScaler
cod_nominal = OneHotEncoder(sparse=False)
cod_nominal.fit_transform(data[['ocean_proximity']])

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

#creo una lista de varibles categoricas y otra de variables numéricas
num_list = ['median_income', 'total_rooms', 'housing_median_age','bed_rooms_per_room']
cat_list = ['ocean_proximity']

#defino el pipeline para datos numerico y categóricos. Solo utilizo un paso: estandarizacion de datos para numericos y encoding para las categóricas
num_transform = Pipeline(steps= [('scaler', StandardScaler())])
cat_transform = Pipeline(steps= [('onehot', OneHotEncoder())])

#Utilizo ColumnTransformer
#Permite la transformación de datos de forma selectiva
transformer = ColumnTransformer([
    ('num',num_transform,num_list),
    ('cat',cat_transform,cat_list)
])

#Separo datos y variable objetivo
X = data.drop('median_house_value', axis=1)
y = data['median_house_value']

#Divido los datos en train y test
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.7)

#Reviso tamaño de las muestras
X_train.shape,X_test.shape,y_train.shape,y_test.shape

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


((14303, 10), (6130, 10), (14303,), (6130,))

5. Prueba de modelo: **Regresión Lineal**


In [ ]:
from sklearn.linear_model import LinearRegression
linear_regression = LinearRegression()

#pipeline de la predicción completa
model_linear_regression = Pipeline([
    ('transformer', transformer),
    ('linar_regressor', linear_regression)
])

#realizó el entrenmiento con los datos de train
model_linear_regression.fit(X_train,y_train)

#voy a realizar una prueba de prediccion con 10 instancias
#para eso uso datos de test, los cuales el modelo no conoce
muestra_de_datos = X_test.iloc[:10]

#le pido al modelo la prediccion de esos registros
model_linear_regression.predict(muestra_de_datos)

#compraro con los valores reales
np.array(y_test.iloc[:10])

#evaluo el modelo usando CROSSVALIDATION
from sklearn.model_selection import cross_val_score
linear_scores = cross_val_score (model_linear_regression, X,y, scoring='neg_mean_squared_error', cv=10)
linear_scores

#hay que cambiar el signo del resultado anterior y sacar la raiz cuadrada
#desta manera se calcula el error medio cuadrático
linear_rmse_scores = np.sqrt(-linear_scores)
linear_rmse_scores

#defino una funcion que muestra  la media y la desviacion standard
def display_scores(scores):
    print('Scores: ', scores)
    print('Media: ', scores.mean())
    print('Desviación Estandard: ', scores.std())
display_scores(linear_rmse_scores)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:839: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 136, in __call__
    score = scorer._score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 353, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 86, in _cached_call
    result, _ = _get_response_values(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_response.py", line 218, in _get_response_values
    y_pred, pos_label = estimator.predict(X), None
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 514, in predict
    Xt = transform.transform(Xt)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils

Scores:  [87394.80828634 60201.19395636 91674.33089871 70363.5334771
            nan 70359.49855267 54945.67304355 92862.40810597
 83361.00198491 54985.31282018]
Media:  nan
Desviación Estandard:  nan


6. Arboles de decición.

In [ ]:
#Arboles de decición
from sklearn.tree import DecisionTreeRegressor

#vuelvo a realizar los mismos pasos que con Regresion Lineal
#1)nombro una variable para nuestro modelo
tree_reg = DecisionTreeRegressor(random_state=42)

#2)defino el Pipeline: Transformación + modelo
model_tree_reg = Pipeline(steps=[
    ('transformer', transformer),
    ('tree_reg', tree_reg)
])

#3)realiezo el entrenamiento
model_tree_reg.fit(X_train,y_train)

#evaluo los scores del modelo, igual que anteriormente con Reg. Lineal
#4) calculo el score con la validación cruzada cv=10
tree_reg_scores = cross_val_score(model_tree_reg, X,y, scoring='neg_mean_squared_error', cv=10)

#5) realizo el cambio de signo y saco la raiz cuadrada
tree_rsme_scores = np.sqrt(-tree_reg_scores)

#6) muestro resltado con la funcion display_scores
display_scores(tree_rsme_scores)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:839: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 136, in __call__
    score = scorer._score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 353, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 86, in _cached_call
    result, _ = _get_response_values(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_response.py", line 218, in _get_response_values
    y_pred, pos_label = estimator.predict(X), None
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 514, in predict
    Xt = transform.transform(Xt)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils

Scores:  [105556.93810156  78112.22041964 115878.51248801 103011.53477862
             nan  92373.99335328  73240.93261149 108583.36581058
 106916.93011481  78043.6836567 ]
Media:  nan
Desviación Estandard:  nan


7. RandomForest

In [ ]:
#RandomForest
from sklearn.ensemble import RandomForestRegressor

#vuelvo a realizar los mismos pasos que con Regresion Lineal y con Arboles de decisión

#1) nombro una variable para el modelo
forest_reg = RandomForestRegressor()

#2)defino el pipeline: transformación + modelo
model_forest_reg = Pipeline(steps=[
    ('transformer', transformer),
    ('forest_reg', forest_reg)
])

#3) ajusto el modelo - entreno
model_forest_reg.fit(X_train,y_train)

#Evaluación
#4) calculo el score con la validación cruzada cv=10
forest_reg_scores = cross_val_score(model_forest_reg,X,y,scoring='neg_mean_squared_error',cv=10)

#5) realizo el cambio de signo y saco la raiz cuadrada
forest_rsme_score = np.sqrt(-forest_reg_scores)

#6) muestro resltado con la funcion display_scores
display_scores(forest_rsme_score)


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:839: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 136, in __call__
    score = scorer._score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 353, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 86, in _cached_call
    result, _ = _get_response_values(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_response.py", line 218, in _get_response_values
    y_pred, pos_label = estimator.predict(X), None
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 514, in predict
    Xt = transform.transform(Xt)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils

Scores:  [78456.04809662 57205.38633575 84096.87986903 74458.93645254
            nan 69742.67765702 51492.74947376 84125.78577499
 81322.21812245 56563.61938856]
Media:  nan
Desviación Estandard:  nan


8. Resumen de los entrenamientos

In [ ]:
#Resumen de los entenamientos
modelos = ['Regresion Lineal', 'Arboles de decisión', 'RandomForest']
media = [75438.78, 95947.18, 74754.12]
desviacion_std = [14309.77 ,14254.19, 11741.90]

resumen = pd.DataFrame()

resumen['Modelos'] = modelos
resumen['Media de Error cuadratico'] = media
resumen['Desviacion estandard'] = desviacion_std

resumen

,Modelos,Media de Error cuadratico,Desviacion estandard
0,Regresion Lineal,75438.78,14309.77
1,Arboles de decisión,95947.18,14254.19
2,RandomForest,74754.12,11741.90


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['Media de Error cuadratico'].plot(kind='hist', bins=20, title='Media de Error cuadratico')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['Desviacion estandard'].plot(kind='hist', bins=20, title='Desviacion estandard')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_3.groupby('Modelos').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
_df_10['Media de Error cuadratico'].plot(kind='line', figsize=(8, 4), title='Media de Error cuadratico')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_11['Desviacion estandard'].plot(kind='line', figsize=(8, 4), title='Desviacion estandard')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_12['Modelos'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_12, x='index', y='Modelos', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_13['Modelos'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_13, x='Media de Error cuadratico', y='Modelos', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_14['Modelos'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_14, x='Desviacion estandard', y='Modelos', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

Podemos ver que el modelo que obtiene la menor media es el RandomForest y a su vez es que obtuvo la menor desviación estándard.